Ensure pat2vec is on path

In [ ]:
pwd

In [ ]:
import sys
sys.path.insert(0,'/home/aliencat/samora/gloabl_files')
sys.path.insert(0,'/data/AS/Samora/gloabl_files')
sys.path.insert(0,'/home/jovyan/work/gloabl_files')
sys.path.insert(0,'/home/cogstack/samora/_data/gloabl_files')

In [ ]:
from pat2vec.util.config_pat2vec import config_class
from datetime import datetime

from post_processing import process_csv_files
from post_processing import extract_datetime_to_column
from pat2vec.pat2vec_pat_list.get_patient_treatment_list import get_all_patients_list
from pat2vec.util.post_processing import produce_filtered_annotation_dataframe

import pandas as pd

### Set options
main_options_dict = {'demo':True,
                'bmi':True,
                'bloods':True,
                'drugs':True,
                'diagnostics':True,
                
                'core_02':True,
                'bed':True,
                'vte_status':True,
                'hosp_site':True,
                'core_resus':True,
                'news':True,
                
                'smoking':True,
                'annotations':True,
                'annotations_mrc': True,
                'negated_presence_annotations':False
                
               }

annot_filter_arguments = {
    'acc': 0.8,
    'types': ['qualifier value', 'procedure', 'substance', 'finding', 'environment', 'disorder', 'observable entity'],
    'Time_Value': ['Recent', 'Past'],  # Specify the values you want to include in a list
    'Time_Confidence': 0.8,  # Specify the confidence threshold as a float
    'Presence_Value': ['True'],  # Specify the values you want to include in a list
    'Presence_Confidence': 0.8,  # Specify the confidence threshold as a float
    'Subject_Value': ['Patient'],  # Specify the values you want to include in a list
    'Subject_Confidence': 0.8  # Specify the confidence threshold as a float
}

## Init config obj
config_obj = config_class(remote_dump=False,
                 suffix='',
                 treatment_doc_filename='treatment_docs.csv',
                 treatment_control_ratio_n=1,
                 proj_name='new_project',
                 current_path_dir="",
                 main_options = main_options_dict,
                 start_date = (datetime(2020, 1, 1)),
                 years = 0,
                 months = 0, 
                 days = 2,
                 dgx = False, 
                 dhcap = False,
                 dhcap02 = True,
                 batch_mode= True,
                 store_annot = True,
                 share_sftp = True,
                 multi_process = False,
                 annot_first = False,
                 strip_list = True,
                 verbosity = 4,
                 random_seed_val =42,
                 testing = True,
                 use_controls = False,
                 medcat = True,
                 start_time = datetime.now(),
                 patient_id_column_name = 'auto',
                 annot_filter_options = annot_filter_arguments,
                 global_start_year = 1995, 
                 global_start_month = 1,
                 global_end_year = 2023,
                 global_end_month = 1,
                 shuffle_pat_list = False
                 
                 )




In [ ]:
from pat2vec.main_pat2vec import main

In [ ]:
pat2vec_obj = main( cogstack=True, use_filter=False,
             json_filter_path = None, random_seed_val=42, 
             hostname =None, config_obj= config_obj, )


View patient list

In [ ]:
pat2vec_obj.all_patient_list

Make pat vectors for pat 0

In [ ]:
pat2vec_obj.pat_maker(0)

In [ ]:

input_directory = 'new_project/current_pat_lines_parts'
output_csv_file = 'output_file.csv'


process_csv_files(input_directory, out_folder='outputs', output_filename_suffix=output_csv_file, part_size=336)



In [ ]:
df = pd.read_csv(output_csv_file)

In [ ]:
df

In [ ]:



df = extract_datetime_to_column(df)

### Filter the annotation batches by a snomed cui and its related codes. 

In [ ]:
from snomed_methods import snomed_methods_v1
snomed_relations_obj = snomed_methods_v1.snomed_relations(medcat=True)

outcome_variable_cui_for_filter = '40733004'  # infection

print(outcome_variable_cui_for_filter)

filter_root_cui = outcome_variable_cui_for_filter
print(filter_root_cui)

retrieved_codes_snomed_tree, retrieved_names_snomed_tree = snomed_relations_obj.recursive_code_expansion(filter_root_cui, n_recursion = 3, debug=False)

retrieved_codes_snomed_tree[0:5], len(retrieved_codes_snomed_tree), len(retrieved_names_snomed_tree)





In [ ]:
retrieved_names_snomed_tree[0:10]

In [ ]:
retrieved_codes_medcat_cdb, retrieved_names_medcat_cdb  = snomed_relations_obj.get_medcat_cdb_most_similar(filter_root_cui, context_type = 'xxxlong', type_id_filter=[], topn=25)

In [ ]:
retrieved_names_medcat_cdb[0:10]

In [ ]:
all_names_list = list(set(retrieved_names_medcat_cdb + retrieved_names_snomed_tree))

all_codes_list = list(set(retrieved_codes_medcat_cdb + retrieved_codes_snomed_tree))

print(len(all_names_list))

In [ ]:


all_pat_list_ = get_all_patients_list(config_obj=pat2vec_obj.config_obj)


all_annot_filtered_df = produce_filtered_annotation_dataframe(cui_filter=True, meta_annot_filter=True, pat_list=all_pat_list_, config_obj=pat2vec_obj.config_obj, filter_custom_args=pat2vec_obj.config_obj.annot_filter_options, cui_code_list=all_codes_list)

In [ ]:
all_annot_filtered_df